In [ ]:
# %pip install pydot
# %pip install graphviz

In [ ]:
import csv

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt


In [ ]:
dataset_path = 'datasets/keypoint.csv'
class_path = 'datasets/classes.csv'

model_save_path = 'models/keypoint_classifier.hdf5'
tflite_save_path = 'models/keypoint_classifier.tflite'


In [ ]:
df_labels = pd.read_csv(class_path, header=None)
df_data = pd.read_csv(dataset_path, header=None)

class_size = df_labels.shape[0]

In [ ]:
df_data.head()

In [ ]:
sns.countplot(x=df_data[0])

In [ ]:
X = np.loadtxt(dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 2)))
y = np.loadtxt(dataset_path, delimiter=',', dtype='int32', usecols=(0))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(((21 * 2) + 1, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(class_size, activation='softmax')
])

In [ ]:
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
model.summary()

In [ ]:
# Checkpoint saving
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)
print(val_loss, val_acc)

In [ ]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
labels = sorted(list(set(y_test)))
y_pred = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred, labels=labels)

df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, fmt='d', square=False)

In [ ]:
print('Classification Report:', classification_report(y_test, y_pred))

In [ ]:
model.save(model_save_path, include_optimizer=False)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

In [ ]:
open(tflite_save_path, 'wb').write(tflite_quantized_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

In [ ]:
model = tf.keras.models.load_model(model_save_path)